<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1b_S6_data_access_SFTP.ipynb"><< Accessing Sentinel-6 data via Data Store</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_S6_coverage.ipynb">Determining Sentinel-6 product coverage >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn Sentinel-6: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook:
- **<a href="1_1a_S6_data_access_SFTP.ipynb" target="_blank">1_1a_S6_data_access_SFTP</a>**; this will download the required Sentinel-6 products for this notebook.

</div>
<hr>

# 1.2 Understanding Sentinel-6 product structure

### Data used

| Product Description  | Data Store collection ID| Product Navigator | 
|:--------------------:|:-----------------------:|:-------------:|
| Sentinel-6 Poseidon-4 Altimetry level-1B Low Resolution (NTC) | EO:EUM:DAT:0118 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0118?query=Altimetry%20NTC&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| Sentinel-6 Poseidon-4 Altimetry level-2 Low Resolution (NTC) | EO:EUM:DAT:0120 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0120?query=Altimetry%20NTC&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| Sentinel-6 Poseidon-4 Altimetry level-1B High Resolution (NTC) | EO:EUM:DAT:0126 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0126?query=Altimetry%20NTC&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| Sentinel-6 Poseidon-4 Altimetry level-2 High Resolution (NTC) | EO:EUM:DAT:0128 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0128?query=Altimetry%20NTC&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| Sentinel-6 Poseidon-4 Altimetry level-2P High Resolution (NRT) | EO:EUM:DAT:0134 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0134?query=Altimetry%20STC%202P&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| Sentinel-6 Poseidon-4 Altimetry level-3 High Resolution (STC) | EO:EUM:DAT:0139 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0139?query=Wave%20STC%202P&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |


### Learning outcomes

At the end of this notebook you will know;
* What the SAFE format is
* What components are inside a SAFE format file from Sentinel-6
* What variables are in present in each component for Sentinel-6 data files (level-1B, level-2, level-2P and level-3)

### Outline

Data from all the Sentinel satellites operated under the European Commissions Copernicus Programme are delievered in "**SAFE format**". The Sentinel-SAFE format is a specific variation of the Standard Archive Format for Europe (SAFE) format specification designed for the Sentinel satellite products.  It is based on the XML Formatted Data Units (XFDU) standard under development by the Consultative Committee for Space Data Systems (CCSDS). Sentinel-SAFE is a profile of XFDU, and it restricts the XFDU specifications for specific utilisation in the Earth Observation domain, providing semantics in the same domain to improve interoperability between ground segment facilities.


Each product package includes:

* a manifest file containing a metadata section and a data object section (an xml file).

* measurement data files (NetCDF-4 format)

* annotation data files, if defined (NetCDF-4 format)

The product package can exist as a directory in a filesystem, zipped folder or tarball.

The naming of the Sentinel-SAFE files follows a specific convention that you can learn about <a href="https://eumetsatspace.atlassian.net/wiki/spaces/DPF/pages/edit-v2/1597702263?" target="_blank">here</a>

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Querying Sentinel-6 file structure for a level-1B **low**-resolution product](#section1)
 2. [Querying Sentinel-6 file structure for a level-1B **high**-resolution product](#section2)
 3. [Applying your knowledge: opening level-2, level-2P and level-3 files](#section3)

<hr>

In [1]:
# library imports
import glob
import inspect
from IPython.display import display, Markdown
import os
import netCDF4 as nc
import xarray as xr
import xml.etree.ElementTree as ET
import warnings

warnings.filterwarnings('ignore')

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Querying Sentinel-6 file structure for a level-1B low-resolution product
[Back to top](#TOC_TOP)

</div>

First we will create a variable which holds the paths for the directories of the products that we downloaded in the **<a href="1_1a_S6_data_access_SFTP.ipynb" target="_blank">1_1a_S6_data_access_SFTP</a>** notebook.

In [2]:
SAFE_directories = glob.glob(os.path.join(os.getcwd(), 'products','*'))

Now lets set a key to match to the product we are interested in. For a level-1B low resolution (LR) Sentinel-6 file, we will set the key as;

In [3]:
match_key = "S6A_P4_1B_LR"

Next, we'll cycle through our products until we find the matching key and read find the manifest file in the SAFE format directory.  

In [4]:
for SAFE_directory in SAFE_directories:
    if match_key in SAFE_directory:
        SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
        display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))
        break

**Manifest file:** xfdumanifest.xml

Then, we'll read the manifest file in to a Python object format called a dictionary. This will make it easier for us to interact with the product in the next step.

In [5]:
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*File info: {}*".format(item[0][0].get('textInfo'))))
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

**File name:** measurement.nc

*File info: Level 1 Product*

The next section of code will display each of the components within the product. Scrolling through you can see the main groups of variables, including some you may already know or be interested in working with. It is important to note that the variables are stored in a series of "groups" in the netCDF file. We need to cycle through these groups to open them in xarray (the library we use to work with netCDF files) and see the **variables**. We will also open the "top" group, which contains the data **coordinates**.

In [6]:
for SAFE_component in sorted(SAFE_components):
    nc_fid = nc.Dataset(SAFE_component)
    groups = []
    groups.append(None)
    for item in nc_fid.groups:
        groups.append(f"{item}/{list(nc_fid.groups[item].groups.keys())[0]}")
    nc_fid.close()
    for group in groups:
        print(f"Group: {group}")
        ds = xr.open_dataset(SAFE_component, group=group, decode_times=False)
        display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
        display(ds)
        ds.close()

Group: None


**Filename:** measurement.nc

<xarray.Dataset>
Dimensions:   (samples: 256, space_3d: 3)
Coordinates:
  * samples   (samples) int16 0 1 2 3 4 5 6 7 ... 249 250 251 252 253 254 255
  * space_3d  (space_3d) int8 1 2 3
Data variables:
    *empty*
Attributes: (12/47)
    Convention:                             CF-1.7
    title:                                  Altimeter L1B LR Non Time Critical
    institution:                            EUMETSAT
    source:                                 Processing Baseline F07
    history:                                2022-09-21 10:44:13 : Creation
    references:                             Sentinel-6/Jason-CS ALT Level 1 P...
    ...                                     ...
    xref_range_cor_internal_delay_cal_ku:   S6A_P4_1__ECHO_AX_20170305T064353...
    xref_attenuator_table_ku:               S6A_P4_1__CHAR_AX_20151008T000000...
    xref_cal2_correction_c:                 None
    xref_altimeter_power_drift_c:           S6A_P4_1__C1HR_AX_20170305T064143...
    xref_range_cor_internal_delay_cal_c:    S6A_P4_1__C1HR_AX_20170305T064143...
    xref_attenuator_table_c:                S6A_P4_1__CHAR_AX_20151008T000000...

Group: global/ku


**Filename:** measurement.nc

<xarray.Dataset>
Dimensions:                          (samples: 256)
Dimensions without coordinates: samples
Data variables:
    antenna_gain                     float64 ...
    cal2_correction                  (samples) float64 ...
    g_scaling                        float64 ...
    processing_configuration_flags   float32 ...
    range_cor_external_group_delay   float64 ...
    range_oversampling_factor        float32 ...
    residual_fixed_digital_gain_lrm  float64 ...

Group: data_20/ku


**Filename:** measurement.nc

<xarray.Dataset>
Dimensions:                            (time: 66977, space_3d: 3, samples: 256)
Coordinates:
    latitude                           (time) float64 ...
    longitude                          (time) float64 ...
  * time                               (time) float64 7.153e+08 ... 7.153e+08
Dimensions without coordinates: space_3d, samples
Data variables: (12/40)
    altimeter_clock                    (time) float64 ...
    altimeter_power_drift              (time) float32 ...
    altitude                           (time) float64 ...
    altitude_rate                      (time) float64 ...
    attenuation_calibrated             (time) float32 ...
    cal1_power                         (time) float32 ...
    ...                                 ...
    tm_source_sequence_counter         (time) float32 ...
    tracker_range_calibrated           (time) float64 ...
    tracker_range_calibrated_diode     (time) float64 ...
    variable_digital_gain              (time) float32 ...
    velocity_vector                    (time, space_3d) float64 ...
    waveform_scale_factor              (time) float32 ...

Here we can see that the data is stored in two groups; `global/ku` and `data/ku`. The former contains the information related to the measurements. The latter contains the measurements themselves, including the waveforms. We can also see that waveform variable is two dimensional `(time, samples)` (you may need to expand the `data/ku` *Data variables* tag to see this. Compare these dimensions to the ones used in the high resolution product in the next section.

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Querying Sentinel-6 file structure for a level-1B high-resolution product
[Back to top](#TOC_TOP)

</div>

Now let's look at the level-1B **high**-resolution Sentinel-6 product that we downloaded previously. We can do this using exactly the same routines as above, but swapping the match key as follows.

In [7]:
match_key = "S6A_P4_1B_HR"

The box below just consolidates all the code snippets from above so that you can run it in one go.

In [8]:
for SAFE_directory in SAFE_directories:
    if match_key in SAFE_directory:
        SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
        display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))
        break

# Reading SAFE manifest and outputting/collecting component names
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*File info: {}*".format(item[0][0].get('textInfo'))))
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

# Display component structure and variables
for SAFE_component in sorted(SAFE_components):
    nc_fid = nc.Dataset(SAFE_component)
    groups = []
    groups.append(None)
    for item in nc_fid.groups:
        groups.append(f"{item}/{list(nc_fid.groups[item].groups.keys())[0]}")
    nc_fid.close()
    for group in groups:
        print(f"Group: {group}")
        ds = xr.open_dataset(SAFE_component, group=group, decode_times=False)
        display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
        display(ds)
        ds.close()

**Manifest file:** xfdumanifest.xml

**File name:** measurement.nc

*File info: Level 1 Product*

Group: None


**Filename:** measurement.nc

...

**Filename:** measurement.nc

<xarray.Dataset>
Dimensions:                            (pulses: 64, samples: 256)
Dimensions without coordinates: pulses, samples
Data variables:
    antenna_gain                       float64 ...
    burst_phase_array_cor              (pulses) float64 ...
    burst_power_array_cor              (pulses) float64 ...
    cal2_correction                    (samples) float64 ...
    g_scaling                          float64 ...
    hr_processing_configuration_flags  float32 ...
    processing_configuration_flags     float32 ...
    range_cor_external_group_delay     float64 ...
    range_oversampling_factor          float32 ...
    residual_fixed_digital_gain_raw    float64 ...
    residual_fixed_digital_gain_rmc    float64 ...

Group: data_20/ku


**Filename:** measurement.nc

<xarray.Dataset>
Dimensions:                            (time: 63376, space_3d: 3,
                                        samples_ov: 512, looks: 322)
Coordinates:
    latitude                           (time) float64 ...
    longitude                          (time) float64 ...
  * time                               (time) float64 7.153e+08 ... 7.153e+08
Dimensions without coordinates: space_3d, samples_ov, looks
Data variables: (12/56)
    altimeter_clock                    (time) float64 ...
    altimeter_power_drift              (time) float32 ...
    altitude                           (time) float64 ...
    altitude_rate                      (time) float64 ...
    attenuation_calibrated             (time) float32 ...
    cal1_power                         (time) float32 ...
    ...                                 ...
    tm_source_sequence_counter         (time) float32 ...
    tracker_range_calibrated           (time) float64 ...
    tracker_range_calibrated_diode     (time) float64 ...
    variable_digital_gain              (time) float32 ...
    velocity_vector                    (time, space_3d) float64 ...
    waveform_scale_factor              (time) float32 ...

We can see some differences between the LR and HR products. Notably, the `power_waveform` variable has different dimensions to its low resolution counterpart, 512 vs 256 as it is over-sampled in the HR case. We also introduce a new coordinate `looks`, and a new variable `stack_mask_start_stop`, which describe the masking used in "multi-looking".

<div class="alert alert-danger" role="alert">

## <a id='section3'></a>3. Applying your knowledge: opening level-2, level-2P and level-3 files

[Back to top](#TOC_TOP)

</div>

<div class="alert alert-block alert-warning">

### Challenge:

Can you adapt the match_keys above to show the contents of a Level-2, Level-2P and Level-3 file? What differences do you notice?<div>

### Enter your solution here
* ...
* ...

* To assess level-2 LR files the match_key to use is: S6A_P4_2__LR
* To assess level-2 HR files the match_key to use is: S6A_P4_2__HR
* To assess level-2P HR files the match_key to use is: S6A_P4_2P_HR
* To assess level-3 HR files the match_key to use is: S6A_P4_3__HR

* key difference: at level-2, there are two files (RED/STD) for standard and reduced measurements
* key difference: at level-2, the standard and reduced measurements are measured at different frequencies (20 Hz and 1 Hz, respectively)
* key difference: at level-2P and level-3, there are no groups, everything is stored at the same level.

<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1b_S6_data_access_SFTP.ipynb"><< Accessing Sentinel-6 data via Data Store</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_S6_coverage.ipynb">Determining Sentinel-6 product coverage >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>